## **Lectura 1: Simplex para problemas de asignación** 



Supongamos que queremos asignar cuatro tareas a cuatro trabajadores (o procesos a máquinas, o inquilinos a casas, o personas en sitios de citas); en fin, relacionamientos uno a uno. Debemos suponer también que, para cada pareja, existe una medida de qué tan buena (o mala) es esa asignación, por ejemplo: cuánto le toma a cada trabajador o máquina hacer cada una de las tareas (y se quiere minimizar el tiempo), o qué tanto tienen en común dos usuarios del sitio de citas (y se quiere maximizar ese índice).


La formulación primal y dual del problema de asignación es la siguiente:

- Sea $C$ una matriz de $n\times n$ de costos (o beneficios) de asignar la tarea $i$ al trabajador $j$.

- Sea $x_{ij}$ una variable binaria que vale 1 si la tarea $i$ es asignada al trabajador $j$ y 0 en otro caso.

- Sea $u_i$ y $v_j$ variables duales asociadas a las restricciones de que cada trabajador debe hacer una sola tarea y cada tarea debe ser hecha por un solo trabajador, respectivamente.



minimizar 

$$\sum_{i=1}^n\sum_{j=1}^n c_{ij}x_{ij}$$

sujeto a

$$\sum_{j=1}^n x_{ij} = 1 \forall i=1,\ldots,n$$

$$\sum_{i=1}^n x_{ij} = 1 \forall j=1,\ldots,n$$

$$x_{ij} \in \{0,1\} \forall i,j=1,\ldots,n$$



El problema dual es

maximizar 

$$\sum_{i=1}^n u_i + \sum_{j=1}^n v_j$$

sujeto a

$$u_i + v_j \leq c_{ij}  \forall i,j=1,\ldots,n$$

$$u_i \geq 0 \forall i=1,\ldots,n$$

$$v_j \geq 0 \forall j=1,\ldots,n$$




Intentemos aplicar el algoritmo de transporte (dado que podemos ver al problema de asignación como un caso particular de dicho problema). Supongamos que una compañia quiere asignar 4 tareas a un conjunto de 4 maquinas y que el tiempo de alistamiento de las maquinas se muestra en la siguiente tabla:

![](../Images/ASSIGN_.png)

**Solución inicial**

![](../Images/ASSIGN_1.png)


Calculando los costos reducidos de las variables no básicas:

- $\bar{c}_{11} = 11$
- $\bar{c}_{21} = 1$
- $\bar{c}_{22} = 9$
- $\bar{c}_{23} = 0$
- $\bar{c}_{31} = 8$
- $\bar{c}_{32} = 7$
- $\bar{c}_{34} = -1$
- $\bar{c}_{43} = 4$


Dado que tenemos un costo reducido negativo, la solución no es óptima. 

**Iteración del simplex de transporte**

![](../Images/ASSIGN_2.png)

![](../Images/ASSIGN_3.png)

Vemos que: La solución óptima es degenerada

- Aunque necesitamos $2m-1$ variables básicas, para que la solución sea factible se requiere apenas $m$ variables diferentes de cero.


Veamos un ejemplo concreto con máquinas y tareas. Supongamos que tenemos cuatro máquinas, y cuatro tareas. La siguiente tabla muestra el tiempo que le toma a cada máquina hacer cada tarea:    


In [1]:
import pandas as pd

M = ["Maq. "+str(i) for i in range(1,5)]
T = ["Tarea "+str(i) for i in range(1,5)]

# Generar dataframe con costos de asignar cada tarea a cada máquina
asignacion = pd.DataFrame( {T[0]:[14,2,7,2],
                            T[1]:[5,12,8,4],
                            T[2]:[8,6,3,6],
                            T[3]:[7,5,9,10]}, index=M)
asignacion_copia = asignacion.copy()
asignacion

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,14,5,8,7
Maq. 2,2,12,6,5
Maq. 3,7,8,3,9
Maq. 4,2,4,6,10


## 2. Cómo funciona el algoritmo de Kuhn

Si uno hace una búsqueda en Internet, seguramente le dirán que el algoritmo de Kuhn hace lo siguiente:

1. **Inicialización**:
    * Encuentre el mínimo valor de cada fila y réstelo de dicha fila. 
    * Luego, encuentre el mínimo valor de cada (nueva) columna y réstelo de ella.
2. **Verificación de "convergencia"**: 
    * Encuentre el menor número de líneas (horizontales y/o verticales) que permiten tachar todos los ceros que hayan aparecido en la nueva tabla. 
    * Si el número de líneas requerido es menor al número de asignaciones a lograr, entonces vaya al paso 3. 
    * Si, en cambio, hay tantas líneas como asignaciones a lograr, es posible encontrar una asignación factible, ¡y óptima!, entre las celdas que contengan ceros.
3. **Actualización**:
    * Encuentre el menor valor entre las celdas no tachadas por líneas. 
    * Si a ese valor lo llamamos *k*, reste *k* de todas las celdas no cubiertas por líneas,
    * Luego, sume *k* a aquellas celdas donde se crucen líneas.
    * Vuelva al paso 2.



Veamos esto en nuestro ejemplo: El problema consiste en elegir un emparejamiento uno a uno de máquina y tarea que represente el menor costo posible; por ejemplo, en terminos de tiempo, si los valores de la tabla corresponden a tiempos que toman las tareas según la máquina.


In [2]:
# Encontrar mínimo de cada fila
asignacion["min_cada_fila"] = asignacion.min(axis=1)
asignacion

,Tarea 1,Tarea 2,Tarea 3,Tarea 4,min_cada_fila
Maq. 1,14,5,8,7,5
Maq. 2,2,12,6,5,2
Maq. 3,7,8,3,9,3
Maq. 4,2,4,6,10,2


In [3]:
# Restar a cada fila su mínimo (por practicidad hago la operación por columnas, pero la operación es en filas)
for i in range(4):
    asignacion.iloc[:,i] = asignacion.iloc[:,i].subtract(asignacion["min_cada_fila"])
asignacion

,Tarea 1,Tarea 2,Tarea 3,Tarea 4,min_cada_fila
Maq. 1,9,0,3,2,5
Maq. 2,0,10,4,3,2
Maq. 3,4,5,0,6,3
Maq. 4,0,2,4,8,2


In [4]:
import numpy as np

# encontrar mínimo de cada (nueva) columna
min_cols = asignacion.min()

# registrarlo
nueva_fila = pd.DataFrame( dict(zip(T+["min_cada_fila"], min_cols)), index=["min_cada_col"] )
nueva_fila.iloc[0,4] = np.nan

asignacion = asignacion.append(nueva_fila)
asignacion

,Tarea 1,Tarea 2,Tarea 3,Tarea 4,min_cada_fila
Maq. 1,9,0,3,2,5.0
Maq. 2,0,10,4,3,2.0
Maq. 3,4,5,0,6,3.0
Maq. 4,0,2,4,8,2.0
min_cada_col,0,0,0,2,NaN


In [5]:
# restar a cada columna su mínimo
for i in range(4):
    asignacion.iloc[0:3,i] = asignacion.iloc[0:3,i] - asignacion.iloc[4,i]

# eliminar columnas agregadas antes
asignacion.drop("min_cada_fila", inplace=True, axis=1)
asignacion.drop("min_cada_col", inplace=True)

asignacion

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,9,0,3,0
Maq. 2,0,10,4,1
Maq. 3,4,5,0,4
Maq. 4,0,2,4,8


In [6]:
import resaltar

# Tachar ceros con MÍNIMA cantidad de líneas horizontales y verticales
asignacion.style.apply(resaltar.tabla, filas=[0,2], cols=[0], col_names=asignacion.columns)

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,9,0,3,0
Maq. 2,0,10,4,1
Maq. 3,4,5,0,4
Maq. 4,0,2,4,8


In [7]:
# Dado que hay 3 líneas tachadas, y se deben hacer 4 asignaciones, NO hay convergencia aun...

# Encontrar mínimo de celdas no cubiertas y llamarlo k
k = min( asignacion.iloc[[1,3],[1,2,3]].min() )
print("Mínimo no cubierto es k="+str(k))

# Restar k de celdas no cubiertas (encontradas por inspección visual)
asignacion.iloc[[1,3],[1,2,3]] = asignacion.iloc[[1,3],[1,2,3]].apply(lambda x: x-k)

# Sumar k a celdas cubiertas dos veces (encontradas por inspección visual)
asignacion.iloc[0,0] = asignacion.iloc[0,0] + k
asignacion.iloc[2,0] = asignacion.iloc[2,0] + k

asignacion.style.apply(resaltar.celdas, cells=[(0,0),(2,0),(1,3)], col_names=asignacion.columns)

Mínimo no cubierto es k=1


,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,10,0,3,0
Maq. 2,0,9,3,0
Maq. 3,5,5,0,4
Maq. 4,0,1,3,7


Cambiaron varios datos en la tabla:
* El 10, 4, 1 de la Maq. 2 pasó a ser 9, 3, 0
* El 2, 4, 8 de la Maq. 4 pasó a ser 1, 3, 7
* El 9 y 4 de los cruces pasaron a 10 y 5

De destacar: hay un nuevo cero.

In [8]:
# Tachar ceros con MÍNIMA cantidad de líneas horizontales y verticales
asignacion.style.apply(resaltar.tabla, filas=[0,2], cols=[0,3], col_names=asignacion.columns)

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,10,0,3,0
Maq. 2,0,9,3,0
Maq. 3,5,5,0,4
Maq. 4,0,1,3,7


Hay 4 líneas y se requieren 4 asignaciones. Si hago asignaciones donde hay ceros, dicha asignación será óptima. Solo es necesario elegir celdas factibles (no se asigna una tarea a más de una máquina, o una máquina a más de una tarea).

In [9]:
asignacion.style.apply(resaltar.celdas, cells=[(0,1),(1,3),(2,2),(3,0)], col_names=asignacion.columns)

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,10,0,3,0
Maq. 2,0,9,3,0
Maq. 3,5,5,0,4
Maq. 4,0,1,3,7


Se ignoran los demás ceros puesto que se incurriría en dobles asignaciones. Sin embargo, se puede garantizar que la asignación hecha es ¡óptima!

¿Es en serio? La asignación implica los siguientes costos, si nos devolvemos a la tabla original:

In [10]:
# Revisar los costos de esa asignación sobre los costos originales del problema
asignacion_copia.style.apply(resaltar.celdas, cells=[(0,1),(1,3),(2,2),(3,0)], col_names=asignacion.columns)

,Tarea 1,Tarea 2,Tarea 3,Tarea 4
Maq. 1,14,5,8,7
Maq. 2,2,12,6,5
Maq. 3,7,8,3,9
Maq. 4,2,4,6,10


Lo anterior conlleva a que el costo total de asignar esas tareas a esas máquinas sea de $15.

Los reto a encontrar una asignación de menor costo. No la van a encontrar. Pero ¿por qué?

## 3. Qué ocurre detrás del algoritmo para decir que es exacto

El problema descrito se puede escribir matemáticamente como:

$$\min \sum\limits_{i\in\mathcal{O}}\sum\limits_{j\in\mathcal{D}}c_{ij}x_{ij}$$


$$\sum\limits_{j\in\mathcal{D}}x_{ij} = 1, \forall i \in \mathcal{O}$$

$$\sum\limits_{i\in\mathcal{O}}x_{ij} = 1, \forall j \in \mathcal{D}$$

$$x_{ij} \geq 0, \forall i\in \mathcal{O},j\in \mathcal{D}$$

Cuyo problema dual (relajando la integralidad de las variables) es:

$$\max \sum\limits_{i\in \mathcal{O}}u_i + \sum\limits_{j\in \mathcal{D}}v_j$$

$$ u_i + v_j \leq c_{ij}, \forall i\in \mathcal{O},j\in \mathcal{D}$$

$$ u_i \in \mathbb{R}, \forall i\in \mathcal{O}$$

$$ v_j \in \mathbb{R}, \forall j\in \mathcal{D}$$



In [11]:
min_filas = list(asignacion_copia.min(axis=1))
min_cols = [0,0,0,2] # mínimos sobre columna, pero después de restar, ver paso XX

duales = pd.DataFrame( {"u(i)":min_filas, "v(j)":min_cols}, index=[1,2,3,4] )
duales

,u(i),v(j)
1,5,0
2,2,0
3,3,0
4,2,2


In [12]:
def actualizar_costos_red(duales):
    # Conjunto de arcos
    arcos = [(i,j) for i in range(1,5) for j in range(1,5)]
    
    # Costos reducidos
    cred = ["c("+str(i)+","+str(j)+")  -  u("+str(i)+")  -  v("+str(j)+")" for i in range(1,5) for j in range(1,5)]

    c = [asignacion_copia.iloc[i-1,j-1] for i in range(1,5) for j in range(1,5)]
    U = [duales.iloc[i-1,0] for i in range(1,5) for j in range(1,5)]
    V = [duales.iloc[j-1,1] for i in range(1,5) for j in range(1,5)]
    v_cred = [c[i]-U[i]-V[i] for i in range(len(c))]

    asignable = [i==0 for i in v_cred]    

    DF = pd.DataFrame({"(i,j)":arcos, 
                       "Valor costo c(i,j)":c, 
                       "Holgura Dual / C.Redu.Primal":cred,
                       "Valor u(i)":U,
                       "Valor v(j)":V,
                       "Result. 'costo red'":v_cred,
                       "Asignable?":asignable})
    DF.set_index("(i,j)", inplace=True)
    return DF

DF = actualizar_costos_red(duales)
colorear = [i for i in range(len(DF["Asignable?"])) if list(DF["Asignable?"])[i]]

DF.style.apply(resaltar.tabla, filas=colorear, cols=[], col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",9,14,5,0
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",True,"c(2,1) - u(2) - v(1)",0,2,2,0
"(2, 2)",False,"c(2,2) - u(2) - v(2)",10,12,2,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",4,6,2,0
"(2, 4)",False,"c(2,4) - u(2) - v(4)",1,5,2,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",4,7,3,0


Con las alternativas actuales de asignación no es posible generar una solución que no asigne más de una tarea a una máquina, o más de una máquina a una tarea.

¡Debemos generar un nuevo cero en la columna Result. 'costo red.'! Es decir, llevar una nueva holgura del dual a 0 para poder que una nueva variable del primal pueda tomar valor.

¿Cómo hacerlo? Podemos cambiar los valores de alguna(s) variables $u_i$ y/o $v_j$ a nuestro antojo, **siempre y cuando garanticemos que:**

* No perdemos ninguno de los ceros que ya tenemos (porque perderíamos posibles asignaciones)
* No se genera ningún valor negativo en "Result 'costo red.'" (porque eso implicaría que hay una holgura negativa en el dual, y que por lo tanto el dual es infactible)

Por ejemplo, podemos llevar a cero el 1 de la fila (2,4) aumentando el valor de $u_2$ de 2 a 3.

In [13]:
duales.iloc[1,0] = 3

DF = actualizar_costos_red(duales)
colorear = [(i,j) for i in range(4,8) for j in [2,4]]

DF.style.apply(resaltar.celdas, cells=colorear, col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",9,14,5,0
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",False,"c(2,1) - u(2) - v(1)",-1,2,3,0
"(2, 2)",False,"c(2,2) - u(2) - v(2)",9,12,3,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",3,6,3,0
"(2, 4)",True,"c(2,4) - u(2) - v(4)",0,5,3,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",4,7,3,0


Al cambiar $u_2$ de 2 a 3 (actualización señalada en verde), los valores de costos reducidos cambian para las celdas que dependen de $u_2$ (mostradas también en amarillo). En resumen:

* Obtuvimos el cero que queríamos en la fila (2,4)
* ¡En la fila (2,1) apareció un valor **negativo**!
    * Perdimos un cero, pero más importante que eso...
    * De dejarlo así, **el problema dual se haría infactible**

¿Cómo arreglarlo? Podríamos hacer que $v_1$, que se resta en la fila problemática (2,1), pase a valer -1 en vez de 0. Con eso:
* Recuperamos factibilidad dual (porque volveríamos a holguras no negativas)
* Recuperaríamos el cero, que por holgura complementaria permitiría asignar valor a la variable primal asociada

In [14]:
duales.iloc[0,1] = -1

DF = actualizar_costos_red(duales)
colorear = [(i,j) for i in [0,4,8,12] for j in [2,5]]

DF.style.apply(resaltar.celdas, cells=colorear, col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",10,14,5,-1
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",True,"c(2,1) - u(2) - v(1)",0,2,3,-1
"(2, 2)",False,"c(2,2) - u(2) - v(2)",9,12,3,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",3,6,3,0
"(2, 4)",True,"c(2,4) - u(2) - v(4)",0,5,3,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",5,7,3,-1


Al cambiar $v_1$ de 0 a -1 (actualización señalada en verde), los valores de costos reducidos cambian para las celdas que dependen de $v_1$ (mostradas también en verde). En resumen:

* Recuperamos la no-negatividad y el cero que queríamos en la fila (2,1)
* Perdimos el cero que había en la fila (4,1); es decir, perdimos una posible asignación

¿Cómo arreglarlo? Podríamos hacer que $u_4$, que se resta en la fila problemática (2,1), pase a valer 3 en vez de 2. Con eso recuperaríamos el cero, que por holgura complementaria permitiría asignar valor a la variable primal asociada.

In [15]:
duales.iloc[3,0] = 3

DF = actualizar_costos_red(duales)
colorear = [(i,j) for i in range(12,16) for j in [2,4]]

DF.style.apply(resaltar.celdas, cells=colorear, col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",10,14,5,-1
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",True,"c(2,1) - u(2) - v(1)",0,2,3,-1
"(2, 2)",False,"c(2,2) - u(2) - v(2)",9,12,3,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",3,6,3,0
"(2, 4)",True,"c(2,4) - u(2) - v(4)",0,5,3,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",5,7,3,-1


¡Hemos recuperado el cero **SIN** generar valores negativos ni perder otros ceros!

Veamos qué asignaciones son posibles ahora, ya que deberíamos tener un cero más, o sea, un asignable más...

In [16]:
colorear = [i for i in range(len(DF["Asignable?"])) if list(DF["Asignable?"])[i]]

DF.style.apply(resaltar.tabla, filas=colorear, cols=[], col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",10,14,5,-1
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",True,"c(2,1) - u(2) - v(1)",0,2,3,-1
"(2, 2)",False,"c(2,2) - u(2) - v(2)",9,12,3,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",3,6,3,0
"(2, 4)",True,"c(2,4) - u(2) - v(4)",0,5,3,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",5,7,3,-1


Sobre esta versión SÍ podemos hacer una asignación factible (que no se asigne más de una máquina a una tarea o viceversa).

In [17]:
colorear = [1,7,10,12]

DF.style.apply(resaltar.tabla, filas=colorear, cols=[], col_names=DF.columns)

,Asignable?,Holgura Dual / C.Redu.Primal,Result. 'costo red',"Valor costo c(i,j)",Valor u(i),Valor v(j)
"(i,j)",,,,,,
"(1, 1)",False,"c(1,1) - u(1) - v(1)",10,14,5,-1
"(1, 2)",True,"c(1,2) - u(1) - v(2)",0,5,5,0
"(1, 3)",False,"c(1,3) - u(1) - v(3)",3,8,5,0
"(1, 4)",True,"c(1,4) - u(1) - v(4)",0,7,5,2
"(2, 1)",True,"c(2,1) - u(2) - v(1)",0,2,3,-1
"(2, 2)",False,"c(2,2) - u(2) - v(2)",9,12,3,0
"(2, 3)",False,"c(2,3) - u(2) - v(3)",3,6,3,0
"(2, 4)",True,"c(2,4) - u(2) - v(4)",0,5,3,2
"(3, 1)",False,"c(3,1) - u(3) - v(1)",5,7,3,-1


¡LA MISMA SOLUCIÓN QUE HABÍAMOS LOGRADO CON LAS REGLAS LOCAS!

Los ceros que no se usan corresponden al típico degeneramiento que presentan los problemas de asignación. La base debe ser de tamaño igual a dos veces el número de nodos ($m$) menos uno, pero para tener una asignación factible, solo $m$ toman valor.



**Observaciones:**

* Cuando se toma la tabla de costos y se restan los mínimos de fila y columna, se están calculando en cada celda los **costos reducidos del problema primal**, que son las mismas holguras del dual, a través de darle valores arbitrarios *pero factibles* a las variables duales (son factibles al evitar que las holguras, o valores de las celdas, se hagan negativas).

* Esa solución es factible para el dual **pero NO para el primal** (o no necesariamente). En efecto, no era posible hacer asignaciones factibles (uno a uno) con las condiciones de holgura complementaria de la primera iteración (es decir, como solo pueden tomar valor las variables relacionadas con holguras duales en cero, no se lograba el uno a uno).

* Conclusión de lo anterior: **el dual era factible pero el primal NO**. No se dejen engañar. Todos los costos reducidos para el problema primal de minimización eran positivos (ya que las holguras duales lo eran), PERO NO se podía decretar optimalidad, ya que **NO** había factibilidad primal; no se cumplían las condiciones KKT.

Cómo opera el algoritmo:
1. Empieza con una solución dual factible ($u_i$ y $v_j$ triviales que no hagan negativos sus $c_{ij}$)
2. Revisa si hay factibilidad primal
    * Si sí, hay optimalidad por KKT
    * Si no, pasa a 3
3. Aumenta el valor de una dual buscando un nuevo cero en holguras (apretar restricciones), PERO:
    * Debe reducir el valor de otra(s) dual(es) para revertir holguras que se hayan negativos
    * Debe aumentar el valor de otra(s) dual(es) más para recuperar ceros perdidos
    * Volver a 2

En el fondo, se hace lo mismo que en Simplex, pero desde la perspectiva del problema dual. Es decir: (i) partir de una solución factible; (ii) verificar costos reducidos (¡que es verificar factibilidad dual!); (iii) moverme buscando factibilidad dual SIN PERDER la primal. Cuando Simplex, desde su factibilidad primal, encuentra factibilidad dual (i.e., costos reducidos que no prometen), entonces decreta optimalidad.

Este tipo de algoritmos se conocen como Simplex-Dual, y desde su factibilidad dual tratan de mejorar su función objetivo; cuando encuentran factibilidad primal, decretan optimalidad.




## Glosario de pre-requisitos

Algunas cosas que vale la pena recordar sobre programación lineal para entender el proceso. Solamente se menciona la idea general de cada caso, pero se recomienda profundizar en ellos si no se dominan.

* *Holgura complementaria*: Si una restricción se cumple con igualdad (i.e., su holgura es cero), entonces la variable asociada a dicha restricción en el problema dual debe tomar valor. Si, por el contrario, una restricción se cumple con desigualdad (i.e., su holgura toma valor), entonces la variable asociada a dicha restricción en el problema dual TIENE que ser cero.

* Los costos reducidos del problema primal son equivalentes a las variables de holgura del problema dual.

* *Condiciones de optimalidad de Karush-Kuhn-Tucker (KKT)*: para garantizar que un problema es óptimo, se debe cumplir:
    * Factibilidad primal
    * Factibilidad dual (o sea costos reducidos "favorables" en el primal)
    * Holgura complementaria

**Ejercicio**

Resuelva usando el algoritmo de Kuhn el siguiente problema de asignación:

El entrenador de la selección colombiana de natación tiene que determinar quién participará en una carrera de 4x100 metros.

![](../Images/ASSIGN_4.png)


¿Cómo debe elegir el entrenador a los miembros del equipo?